In [1]:
!pip install py-entitymatching

     |████████████████████████████████| 2.0MB 3.9MB/s 
     |████████████████████████████████| 788kB 35.5MB/s 
     |████████████████████████████████| 1.1MB 42.5MB/s 
     |████████████████████████████████| 368kB 38.4MB/s 
     |████████████████████████████████| 665kB 40.2MB/s 
  Created wheel for py-entitymatching: filename=py_entitymatching-0.4.0-cp37-cp37m-linux_x86_64.whl size=2634148 sha256=8d42a960512c0de40d6bffa261f9152d3a2a896089fe727122abe07ce9546b6b
  Stored in directory: /root/.cache/pip/wheels/88/65/50/a8bc0f26e8a82c931658153c4f37cbf5debc22cb0f61dbcd05
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp37-cp37m-linux_x86_64.whl size=3723741 sha256=c9681a6c8c893e05c0cb5cf1983adefb74ce1229bd7362542f8eac20302a524a
  Stored in directory: /root/.cache/pip/wheels/58/30/8e/6fa463100ff6e8595b131067ec43a118c607f621ac297e7dad
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linux_x86_64.whl size=2055252 sha256=791efd596569ae9d20d

In [ ]:
# https://hpi.de/naumann/projects/repeatability/datasets/restaurants-dataset.html
# http://www.iesl.cs.umass.edu/datasets.html - CORA dataset
# Importing the libraries
import sys
import pandas as pd
import os
from google.colab import drive
from math import *
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
path_variable = "/content/gdrive/My Drive/BTP/Code/Dataset/Restaurant/"
method = "/content/gdrive/My Drive/BTP/Code/Method3/weights_rest/"

In [ ]:
import py_entitymatching as em
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
pandas version: 1.1.5
magellan version: 0.4.0


In [ ]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = path_variable+"fodors.csv"
path_B = path_variable+"zagats.csv"
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')
# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


Number of tuples in A: 533
Number of tuples in B: 331
Number of tuples in A X B (i.e the cartesian product): 176423


In [ ]:
# verify values in S

d1 = dict()
d2 = dict()

t1 = A.to_numpy()
t2 = B.to_numpy()

for x in t1:
  if(x[-1] in d1):
    d1[x[-1]]+=1
  else:
    d1[x[-1]]=1

for x in t2:
  if(x[-1] in d2):
    d2[x[-1]]+=1
  else:
    d2[x[-1]]=1

print("d1 : ",d1)
print("d2 : ",d2)

d1 :  {'American': 165, 'French/Italian/European': 164, 'Asian': 88, 'Diners/Cafes': 35, 'Fusion food(Mexican/mediterranean/caribbean)': 51, 'bbq/fast food': 30}
d2 :  {'American': 70, 'Asian': 69, 'Fusion food(Mexican/mediterranean/caribbean)': 31, 'bbq/fast food': 47, 'French/Italian/European': 70, 'Diners/Cafes': 44}


In [ ]:
import pickle
import numpy as np
import torch

dbfile = open(path_variable+'files/feature_matrix_restaurant', 'rb')    
cand_file = open(path_variable+'files/C_restaurant', 'rb')  
S = pickle.load(dbfile)
cand = pickle.load(cand_file)


In [ ]:
S

,_id,ltable_id,rtable_id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_sim,addr_addr_jac_qgm_3_qgm_3,addr_addr_cos_dlm_dc0_dlm_dc0,addr_addr_mel,addr_addr_lev_sim,city_city_jac_qgm_3_qgm_3,city_city_cos_dlm_dc0_dlm_dc0,city_city_mel,city_city_lev_sim,type_type_jac_qgm_3_qgm_3,type_type_cos_dlm_dc0_dlm_dc0,type_type_mel,type_type_lev_sim,phone_phone_lev_sim,phone_phone_cos,phone_phone_jac,phone_phone_mel
0,0,555,1,0.000000,0.0,0.622685,0.312500,0.448276,0.577350,0.778992,0.666667,0.125000,0.0,0.678571,0.357143,0.117647,0.0,0.690476,0.375000,0.333333,0.707107,0.545455,0.750000
1,1,663,1,0.088235,0.0,0.627573,0.294118,0.333333,0.500000,0.822015,0.565217,0.083333,0.0,0.688462,0.307692,0.000000,0.0,0.301282,0.076923,0.166667,0.707107,0.545455,0.666667
2,2,677,1,0.051282,0.0,0.589286,0.238095,0.642857,0.750000,0.955556,0.857143,0.083333,0.0,0.688462,0.307692,0.000000,0.0,0.301282,0.076923,0.333333,0.707107,0.545455,0.583333
3,3,549,4,0.111111,0.0,0.583750,0.250000,0.564103,0.670820,0.799818,0.687500,0.103448,0.0,0.669615,0.266667,0.000000,0.0,0.583333,0.222222,0.333333,0.668153,0.500000,0.750000
4,4,665,4,0.000000,0.0,0.484722,0.125000,0.512195,0.670820,0.764642,0.687500,0.103448,0.0,0.669615,0.266667,0.057143,0.0,0.694242,0.277778,0.583333,0.801784,0.666667,0.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4946,4946,764,331,0.045455,0.0,0.548238,0.241379,0.333333,0.577350,0.850826,0.578947,0.000000,0.0,0.414286,0.000000,0.000000,0.0,0.472222,0.125000,0.166667,0.645497,0.454545,0.416667
4947,4947,920,331,0.000000,0.0,0.532471,0.137931,0.653846,0.666667,0.959298,0.850000,1.000000,1.0,1.000000,1.000000,0.000000,0.0,0.640491,0.312500,0.500000,0.577350,0.400000,0.666667
4948,4948,930,331,0.035088,0.0,0.592175,0.172414,0.444444,0.471405,0.807470,0.516129,1.000000,1.0,1.000000,1.000000,0.000000,0.0,0.520833,0.062500,0.416667,0.680414,0.500000,0.583333
4949,4949,945,331,0.068182,0.0,0.658842,0.344828,0.348837,0.235702,0.789774,0.457143,1.000000,1.0,1.000000,1.000000,0.272727,0.0,0.833333,0.500000,0.333333,0.433013,0.272727,0.416667


In [ ]:
cand

,_id,ltable_id,rtable_id,ltable_name,ltable_addr,ltable_city,ltable_phone,ltable_type,ltable_class,ltable_group,rtable_name,rtable_addr,rtable_city,rtable_phone,rtable_type,rtable_class,rtable_group
0,0,555,1,valentino,'3115 pico blvd.','santa monica',310/829-4313,italian,21,French/Italian/European,'apple pan the','10801 w. pico blvd.','west la',310-475-3585,american,534,American
1,1,663,1,'clearwater cafe','168 w. colorado blvd.','los angeles',818/356-0959,'health food',126,bbq/fast food,'apple pan the','10801 w. pico blvd.','west la',310-475-3585,american,534,American
2,2,677,1,'jack sprat\'s grill','10668 w. pico blvd.','los angeles',310/837-6662,'health food',140,bbq/fast food,'apple pan the','10801 w. pico blvd.','west la',310-475-3585,american,534,American
3,3,549,4,'the palm','9001 santa monica blvd.','los angeles',310/550-8811,american,15,American,'belvedere the','9882 little santa monica blvd.','beverly hills',310-788-2306,'pacific new wave',537,Fusion food(Mexican/mediterranean/caribbean)
4,4,665,4,dive!,'10250 santa monica blvd.','los angeles',310/788-,'dive american',128,American,'belvedere the','9882 little santa monica blvd.','beverly hills',310-788-2306,'pacific new wave',537,Fusion food(Mexican/mediterranean/caribbean)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4946,4946,764,331,'dix et sept','181 w. 10th st.','new york',212/645-8023,french,228,French/Italian/European,'ritz-carlton cafe (atlanta)','181 peachtree st.',atlanta,404-659-0400,'american (new)',711,American
4947,4947,920,331,bistango,'1100 peachtree st.',atlanta,404/724-0901,mediterranean,386,Fusion food(Mexican/mediterranean/caribbean),'ritz-carlton cafe (atlanta)','181 peachtree st.',atlanta,404-659-0400,'american (new)',711,American
4948,4948,930,331,'french quarter food shop','923 peachtree st. at 8th st.',atlanta,404/875-2489,southern,396,American,'ritz-carlton cafe (atlanta)','181 peachtree st.',atlanta,404-659-0400,'american (new)',711,American
4949,4949,945,331,'pleasant peasant','555 peachtree st. at linden ave.',atlanta,404/874-3223,american,411,American,'ritz-carlton cafe (atlanta)','181 peachtree st.',atlanta,404-659-0400,'american (new)',711,American


In [ ]:
S = torch.tensor(S.values,dtype=torch.double)
S = S[:,3:]
# S = S.to_numpy(dtype=np.longdouble)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initial weights would be 1/(num_attributes*num_sim_scores) due to add-to-1 constraint
w_init  = (1 / S.shape[1] )
# W = np.full((S.shape[0], S.shape[1]), w_init,dtype=np.longdouble)
W = torch.DoubleTensor(S.shape[1],1).fill_(w_init)

In [ ]:
# ith index of genre_groups denotes genre of ith record pair
type_groups = []
for i in cand['ltable_group']:
    type_groups.append(i)

type_indices = {}
n = len(type_groups)

for i in type_groups:
    type_indices[i] = []

for i in range(len(type_groups)):
    type_indices[type_groups[i]].append(i)

In [ ]:
# ith row of RLS matrix will return RLS value of the ith record pair 
# return nx1 matrix, where n=number of record pairs 
def get_RLS(W,S):
    rls = torch.matmul(S,torch.square(W))
    return rls

# returns groupwise rls_values dictionary
def get_RLS_i(W,S,type_indices):
    rls_vector = get_RLS(W,S) # nx1
    rls_values_i = {}
    for type_ in type_indices:
        rls_values_i[type_] = 0
    for type_ in type_indices:
        indices = type_indices[type_]
        for i in indices:
            rls_values_i[type_] += float(rls_vector[i])
    return rls_values_i

############################################################################################

def get_precomputed_values(W,S,phi,tau):
    numerator = torch.exp(phi*get_RLS(W,S)) - torch.exp(phi* (tau-get_RLS(W,S)))
    # print(numerator)
    denominator = torch.exp(phi*get_RLS(W,S)) + torch.exp(phi* (tau-get_RLS(W,S)))
    return torch.divide(numerator,denominator)

def derivative_loss_function(W,S,j,precomputed_values):
    # print(S[:,j].reshape(-1,1).shape)
    # print(torch.transpose(S[:,j],0,1).shape)
    # print(precomputed_values.shape)
    m = torch.matmul(torch.transpose(S[:,j].reshape(-1,1),0,1),precomputed_values)
    der = -2 * W[j,0] * m
    return der

############################################################################################

def get_sval_i(S,j,indi):
    val = 0
    # print(S.shape)
    for x in indi:
      val += S[x,j]
    return val

def get_fairness_precomputed_value(W,S,type_indices):
    rls_values_sum_dic = get_RLS_i(W,S,type_indices)
    num = np.array([0]*W.shape[0])
    rls_sum_all = torch.sum(get_RLS(W,S)) # sigma RLS_all

    for type_ in type_indices:
        rls_sum_i = rls_values_sum_dic[type_]
        pi =  rls_sum_i / rls_sum_all
        fi = float(len(type_indices[type_])/n)
        
        for j in range(W.shape[0]):
          num[j] += (fi/pi) * (rls_sum_all * get_sval_i(S,j,type_indices[type_]) - rls_sum_i*torch.sum(S[:,j])) 
    # print(num)
    # print((rls_sum_all*rls_sum_all))
    return num / float((rls_sum_all*rls_sum_all)) 

############################################################################################


# Here j is the jth weight
def get_fairness_derivative(W,S,j,precomputed_value_fairness):
    # s1 = torch.transpose(S[j,:].reshape(-1,1),0,1).shape
    # w1 = torch.transpose(W[j,:].reshape(-1,1),0,1).shape


    return -2 * W[j,:] * precomputed_value_fairness[j]


############################################################################################

def algorithm_gradient_descent(W,S,learning_rate,iterations,phi,tau,lambdaa):
    offset = 100
    
    for i in range(iterations):
        W_new = torch.zeros(W.shape) 
        # print(W.shape)
        # print(W_new.shape)

        sum_W_new = 0
        precomputed_values_em = get_precomputed_values(W,S,phi,tau)
        precomputed_value_fairness = get_fairness_precomputed_value(W,S,type_indices)
        
        
        for j in range(W.shape[0]):
            # if j==W.shape[0]-1:
            #     print(get_fairness_derivative(W,S,j,precomputed_value_fairness))

            loss_value = derivative_loss_function(W,S,j,precomputed_values_em) + lambdaa * offset * get_fairness_derivative(W,S,j,precomputed_value_fairness)
            # print(get_fairness_derivative(W,S,j,precomputed_value_fairness))
            # print(derivative_loss_function(W,S,j,precomputed_values_em))
            # print("jay")
            # break
            W_new[j,0] = W[j,0] - learning_rate * loss_value
            sum_W_new += W_new[j,0]

        for j in range(W.shape[0]):
            W_new[j,0] = W_new[j,0] / float(sum_W_new)
        
        W = W_new.double()
        # print(W)
        
        if((i+1)%10==0):
          print("Iteration " + str(i+1) + " completed ")        

    return W

In [ ]:
######### Skew metric adopted from https://arxiv.org/pdf/1905.01989.pdf (refer this paper for the metric)

def fair_calc(weights,S,flag,k):
  
  rls_matrix = get_RLS(weights,S)
  cand['score'] = rls_matrix.numpy()
  candy = cand.sort_values(by=['score'], ascending=False)

  # print(candy)
  # k=500

  d3 = dict()
  d4 = dict()

  temp = candy.to_numpy()
  cnt = 0

  for _id,pair in enumerate(temp):
    # print(_id)
    # print(pair)
    # print(pair,pair[9],pair[-2])
    if(_id>k):
      # print(_id)
      break

    if(pair[9] in d3):
      d3[pair[9]]+=1
    else:
      d3[pair[9]]=1
      
    if(pair[-2] in d4):
      d4[pair[-2]]+=1
    else:
      d4[pair[-2]]=1
    
    if(pair[9]==(pair[-2])):
      cnt+=1

  
  # print("d3 : ", d3)
  # print("d4 : ",d4)

  # print(cnt)
  
  mini = -1
  maxi = 1
  d_skew = {}
  flagy = 0
  d_avg = 0
  for d in d1.keys():
    if(d in d3.keys()):
      d_skew[d] = np.log( d3[d]*len(t1)/(d1[d]*k))
      d_avg += np.abs(d_skew[d])
      if(flagy==0):
        flagy+=1
        mini = d_skew[d]
        maxi = d_skew[d]
      
      mini = min(mini,d_skew[d])
      maxi = max(maxi,d_skew[d])
    # else:
    #   print(d)
  
  if(flag):
    print("Group distribution in result : ",d3)
    print("Skew metric max-min: ", maxi-mini)
    print("Skew metric avg: ", d_avg/len(d1.keys()))
 
  return maxi-mini

In [ ]:
#No learning
print(W)
fair_calc(W,S,True,500)

tensor([[0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500]], dtype=torch.float64)
Group distribution in result :  {'American': 98, 'French/Italian/European': 163, 'bbq/fast food': 45, 'Fusion food(Mexican/mediterranean/caribbean)': 58, 'Asian': 78, 'Diners/Cafes': 59}
Skew metric max-min:  1.0431673776463146
Skew metric avg:  0.30326471263917615


1.0431673776463146

In [ ]:
#No tweak of weights for fairness
# weights corresponding to lambda = 0 

weights0 = algorithm_gradient_descent(W,S,1e-4,100,-100,1,0)
print(weights0)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
tensor([[1.0900e-19],
        [8.4650e-21],
        [9.9498e-07],
        [4.4230e-15],
        [1.9281e-12],
        [4.0534e-09],
        [3.5208e-01],
        [4.1722e-06],
        [2.8003e-10],
        [7.0304e-13],
        [6.4204e-01],
        [6.5846e-07],
        [5.4791e-19],
        [2.5706e-20],
        [1.4682e-06],
        [1.2752e-14],
        [2.0907e-10],
        [2.9796e-03],
        [5.3939e-07],
        [2.8967e-03]], dtype=torch.float64)


In [ ]:
weights1 = algorithm_gradient_descent(W,S,1e-4,100,-100,1,500)
print(weights1)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
tensor([[2.0347e-21],
        [1.1514e-22],
        [2.3588e-07],
        [2.1488e-16],
        [4.7186e-13],
        [1.7748e-09],
        [3.6949e-01],
        [3.8387e-06],
        [1.3353e-10],
        [2.6813e-13],
        [6.2822e-01],
        [4.0242e-07],
        [1.5084e-20],
        [3.4192e-22],
        [4.2692e-07],
        [9.5670e-16],
        [2.8642e-11],
        [1.2379e-03],
        [1.2363e-07],
        [1.0469e-03]], dtype=torch.float64)


In [ ]:
# weights corresponding to lambda= 1000 
weights2 = algorithm_gradient_descent(W,S,1e-4,100,-100,1,1000)
print(weights2)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
tensor([[1.5481e-22],
        [6.8174e-24],
        [9.7067e-08],
        [3.0986e-17],
        [2.2347e-13],
        [1.1386e-09],
        [3.8688e-01],
        [3.8984e-06],
        [1.2111e-10],
        [2.4332e-13],
        [6.1185e-01],
        [3.6476e-07],
        [1.3948e-21],
        [1.8108e-23],
        [2.0260e-07],
        [1.8289e-16],
        [8.2807e-12],
        [7.1286e-04],
        [4.9382e-08],
        [5.5916e-04]], dtype=torch.float64)


In [ ]:
# weights corresponding to lambda= 5000 
weights3 = algorithm_gradient_descent(W,S,1e-4,100,-100,1,5000)
print(weights3)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
tensor([[1.5443e-23],
        [3.4382e-25],
        [7.6821e-08],
        [6.8010e-18],
        [1.6988e-13],
        [1.5882e-09],
        [5.0334e-01],
        [6.5150e-06],
        [8.6102e-11],
        [1.8447e-13],
        [4.9598e-01],
        [2.4923e-07],
        [2.2080e-22],
        [3.0137e-25],
        [6.2736e-07],
        [1.4366e-16],
        [3.0986e-12],
        [4.0219e-04],
        [1.7852e-08],
        [2.7706e-04]], dtype=torch.float64)


In [ ]:
import pickle


dbfile = open(method+'weights0_restaurant', 'wb') 
pickle.dump(weights0, dbfile) 
dbfile.close()

dbfile = open(method+'weights1_restaurant', 'wb') 
pickle.dump(weights1, dbfile) 
dbfile.close()

dbfile = open(method+'weights2_restaurant', 'wb') 
pickle.dump(weights2, dbfile) 
dbfile.close()

dbfile = open(method+'weights3_restaurant', 'wb') 
pickle.dump(weights3, dbfile) 
dbfile.close()


In [ ]:
import pickle
import numpy as np
import torch


wfile = open(method+'weights0_restaurant', 'rb')    
weights0 = pickle.load(wfile)
wfile.close()

wfile = open(method+'weights1_restaurant', 'rb')    
weights1 = pickle.load(wfile)
wfile.close()

wfile = open(method+'weights2_restaurant', 'rb')    
weights2 = pickle.load(wfile)
wfile.close()

wfile = open(method+'weights3_restaurant', 'rb')    
weights3 = pickle.load(wfile)
wfile.close()


In [ ]:
def comp_overlap(w1,w2,k):
  rls_matrix = get_RLS(w1,S)
  cand['score'] = rls_matrix.numpy()
  candy1 = cand.sort_values(by=['score'], ascending=False)

  # print(candy)

  dw1 = dict()
  temp1 = candy1.to_numpy()
  
  rls_matrix = get_RLS(w2,S)
  cand['score'] = rls_matrix.numpy()
  candy2 = cand.sort_values(by=['score'], ascending=False)

  # print(candy)
  dw2 = dict()
  temp2 = candy2.to_numpy()
  overlp = 0
  
  # print(type(candy1),candy1.columns)
  # print(candy2)
  # print(temp1[0])
  # print(temp1[0][1],temp1[0][2])
  # print(temp1)
  # print(temp2)
  # return 
  dname = {}

  
  for _i,i in enumerate(temp1):
    # print(i[1],i[2])
    if(_i>k-1):
      break
    dname[str(i[1])+"#"+str(i[2])] = 1
  
  # print(len(dname))

  for _j,j in enumerate(temp2):
      if(_j>k-1):
        break
      if(str(j[1])+"#"+str(j[2]) in  dname.keys()):
        overlp+=1
  
  return overlp

# comp_overlap(weights1,weights2,500)

In [ ]:

path_G = path_variable+"matches_fodors_zagats.csv"
G = em.read_csv_metadata(path_G, 
                        #  key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='fodors_id', fk_rtable='zagats_id')

# print(G)
print(len(G))

gdic = {}
cnt_g = 0

G1 = G.to_numpy()
for i,x in enumerate(G1):
  s = str(x[0])+"#"+str(x[1])  
  s1 = str(x[1])+"#"+str(x[0])
  # print(s)
  gdic[s] = 1
  gdic[s1] = 1

print("Positive in ground truth",len(gdic)/2)
print(gdic.keys())

def check_gt(w1,S,k):
  rls_matrix = get_RLS(w1,S)
  cand['score'] = rls_matrix.numpy()
  candy1 = cand.sort_values(by=['score'], ascending=False)
  temp1 = candy1.to_numpy()
  # print(temp1)
  overlp = 0
  neg = 0

  for _j,j in enumerate(temp1):
      # print(j)
      if(_j>k-1):
        break
      s = str(j[1])+"#"+str(j[2])

      if(s in  gdic.keys()):
        # print(s)
        overlp+=1

  return overlp

Metadata file is not present in the given path; proceeding to read the csv file.


112
Positive in ground truth 112.0
dict_keys(['534#219', '219#534', '535#220', '220#535', '536#221', '221#536', '537#222', '222#537', '538#223', '223#538', '539#224', '224#539', '540#225', '225#540', '541#226', '226#541', '542#227', '227#542', '543#228', '228#543', '544#229', '229#544', '545#230', '230#545', '546#231', '231#546', '547#232', '232#547', '548#233', '233#548', '549#234', '234#549', '550#235', '235#550', '551#236', '236#551', '552#237', '237#552', '553#238', '238#553', '554#239', '239#554', '555#240', '240#555', '556#241', '241#556', '557#242', '242#557', '558#243', '243#558', '559#244', '244#559', '560#245', '245#560', '561#246', '246#561', '562#247', '247#562', '563#248', '248#563', '564#249', '249#564', '565#250', '250#565', '566#251', '251#566', '567#252', '252#567', '568#253', '253#568', '569#254', '254#569', '570#255', '255#570', '571#256', '256#571', '572#257', '257#572', '573#258', '258#573', '574#259', '259#574', '575#260', '260#575', '576#261', '261#576', '577#262

In [ ]:
print(S.shape)
# print()

for i in [250,500,750,1000]:
  print("\n"+str(i))
  fair_calc(weights0,S,True,i)    #lambda = 0
  print("Overlap with ground truth weights_this", check_gt(weights0,S,i)," /"+ str(len(gdic)/2))

print("####################################################### \n")
for i in [250,500,750,1000]:
  print("\n"+str(i))
  fair_calc(weights1,S,True,i)    #lambda = 500
  print("Overlap with base result = "+str(comp_overlap(weights0,weights1,i))+" /"+ str(i))
  print("Overlap with ground truth weights_this", check_gt(weights1,S,i)," /"+ str(len(gdic)/2))

print("####################################################### \n")
for i in [250,500,750,1000]:
  print("\n"+str(i))
  fair_calc(weights2,S,True,i)    #lambda = 1000  
  print("Overlap with base result = "+str(comp_overlap(weights0,weights2,i))+" /"+ str(i))
  print("Overlap with ground truth weights_this", check_gt(weights2,S,i)," /"+ str(len(gdic)/2))


print("####################################################### \n")
for i in [250,500,750,1000]:
  print("\n"+str(i))
  fair_calc(weights3,S,True,i)    #lambda = 5000
  print("Overlap with base result = "+str(comp_overlap(weights0,weights3,i))+" /"+ str(i))
  print("Overlap with ground truth weights_this", check_gt(weights3,S,i)," /"+ str(len(gdic)/2))


torch.Size([4951, 20])

250
Group distribution in result :  {'American': 42, 'Asian': 43, 'French/Italian/European': 79, 'bbq/fast food': 22, 'Fusion food(Mexican/mediterranean/caribbean)': 33, 'Diners/Cafes': 32}
Skew metric max-min:  1.2786636969275251
Skew metric avg:  0.3524795747397352
Overlap with ground truth weights_this 49  /112.0

500
Group distribution in result :  {'American': 75, 'Asian': 78, 'French/Italian/European': 169, 'bbq/fast food': 45, 'Fusion food(Mexican/mediterranean/caribbean)': 73, 'Diners/Cafes': 61}
Skew metric max-min:  1.3439831630481678
Skew metric avg:  0.39776150099253216
Overlap with ground truth weights_this 55  /112.0

750
Group distribution in result :  {'American': 161, 'Asian': 107, 'French/Italian/European': 252, 'bbq/fast food': 52, 'Fusion food(Mexican/mediterranean/caribbean)': 114, 'Diners/Cafes': 65}
Skew metric max-min:  0.8289139245862873
Skew metric avg:  0.25816109081438077
Overlap with ground truth weights_this 85  /112.0

1000
Group d